In [1]:
pip install ctranslate2


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import ctranslate2

# Load the model from Hugging Face
model_name = "lapp0/flan-t5-small-query-expansion-merged-lr-2e-4-ep-30"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save the model locally
save_dir = "./hf_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Convert the model using CTranslate2
converter = ctranslate2.converters.TransformersConverter(save_dir)
converter.convert(output_dir="optimized_model5", quantization="int8_bfloat16")


c:\Users\shaheryar.ahmed\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\shaheryar.ahmed\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: output directory optimized_model5 already exists, use --force to override

In [26]:
import ctranslate2
import time
# Load the converted model with CTranslate2
translator = ctranslate2.Translator("optimized_model5")

# Prepare input data
input_texts = ["Create a table for top noise cancelling headphones that are not expensive"]
model_name = "lapp0/flan-t5-small-query-expansion-merged-lr-2e-4-ep-30"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input texts
input_encodings = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)

# Record the start time
start_time = time.time()

# Perform inference
output_encodings = model.generate(input_encodings.input_ids)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Decode the output
output_texts = [tokenizer.decode(output_encoding, skip_special_tokens=True) for output_encoding in output_encodings]

for text in output_texts:
    print(text)

# Print the elapsed time
print(f"Elapsed time: {elapsed_time:.4f} seconds")


Top noise cancelling headphones under $100. Top noise cancelling headphones $100 - $200. Best
Elapsed time: 0.6607 seconds


In [35]:
import ctranslate2
import time

# Load the converted model with CTranslate2
translator = ctranslate2.Translator("optimized_model4", device="cpu", inter_threads=4, intra_threads=4)
#model_name = "lapp0/flan-t5-small-query-expansion-merged-lr-2e-4-ep-30"
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare input data
input_texts = ["Who lived longer, Nikola Tesla or Milutin Milankovic?"]

# Tokenize the input texts
input_tokens = [tokenizer.tokenize(text) for text in input_texts]

# Add end-of-sequence token if necessary
input_tokens = [tokens + [tokenizer.eos_token] for tokens in input_tokens]
# Record the start time
start_time = time.time()

# Perform translation
results = translator.translate_batch(
    input_tokens,
    beam_size=1,  # Greedy decoding
    return_scores=False,
    max_decoding_length=24  # Limit length for faster results
)


# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = (end_time - start_time) * 1000

# Decode the output
output_texts = [tokenizer.convert_tokens_to_string(result.hypotheses[0]) for result in results]

for text in output_texts:
    print(text)

# Print the elapsed time
print(f"Elapsed time: {elapsed_time:.4f} milliseconds")

When did Nikola Tesla die. When did Milutin Milankovic die .
Elapsed time: 139.0736 milliseconds


In [42]:
import ctranslate2
import time
from transformers import AutoTokenizer

# Load the converted model with CTranslate2
translator = ctranslate2.Translator("optimized_model4", device="cpu", inter_threads=4, intra_threads=4)
model_name = "lapp0/flan-t5-small-query-expansion-merged-lr-2e-4-ep-30"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Prepare input data
input_texts = ["Who lived longer, Nikola Tesla or Milutin Milankovic?"]

# Tokenize the input texts
input_tokens = [tokenizer.tokenize(text) for text in input_texts]

# Add end-of-sequence token if necessary
input_tokens = [tokens + [tokenizer.eos_token] for tokens in input_tokens]

# Record the start time
start_time = time.time()

# Perform translation
results = translator.translate_batch(
    input_tokens,
    beam_size=1,  # Greedy decoding
    return_scores=False,
    max_decoding_length=24  # Limit length for faster results
)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = (end_time - start_time) * 1000

# Decode the output
output_texts = [tokenizer.convert_tokens_to_string(result.hypotheses[0]) for result in results]

for text in output_texts:
    print(text)

# Print the elapsed time
print(f"Elapsed time: {elapsed_time:.4f} milliseconds")


When did Nikola Tesla die. When did Milutin Milankovic die .
Elapsed time: 170.6848 milliseconds


In [23]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
pip install fastapi uvicorn ctranslate2


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import requests

# Define the API URL
url = "http://127.0.0.1:8000/translate"

# Prepare the input data
input_text = "Create a table for top noise cancelling headphones that are not expensive"
data = {"input_text": input_text}

# Send the request
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    result = response.json()
    print("Output Text:", result["output_text"])
    print("Elapsed Time (ms):", result["elapsed_time"])
else:
    print("Error:", response.status_code, response.text)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /translate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000029E5487ECB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [3]:
import ctranslate2
import time
import os
from transformers import AutoTokenizer
import torch

# Disable gradient computation for inference
torch.set_grad_enabled(False)

# Load optimized model
translator = ctranslate2.Translator(
    "optimized_model4", 
    device="cpu",  
    compute_type="int8",
    inter_threads=os.cpu_count(),
    intra_threads=2
)

model_name = "lapp0/flan-t5-small-query-expansion-merged-lr-2e-4-ep-30"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Tokenization cache
TOKENIZED_CACHE = {}

def get_cached_tokens(text, tokenizer):
    if text not in TOKENIZED_CACHE:
        tokens = tokenizer.tokenize(text)
        tokens = tokens + [tokenizer.eos_token]
        TOKENIZED_CACHE[text] = tokens
    return TOKENIZED_CACHE[text]

# Warm-up (optional but recommended)
warm_up_texts = ["example query"]
warm_up_tokens = [get_cached_tokens(text, tokenizer) for text in warm_up_texts]
translator.translate_batch(warm_up_tokens)

def fast_inference(input_texts):
    # Tokenize inputs
    input_tokens = [get_cached_tokens(text, tokenizer) for text in input_texts]
    
    # Perform translation
    results = translator.translate_batch(
        input_tokens,
        beam_size=1,
        return_scores=False,
        max_decoding_length=16
    )
    
    # Decode outputs
    output_texts = [tokenizer.convert_tokens_to_string(result.hypotheses[0]) for result in results]
    return output_texts

# Main execution
input_texts = ["Who lived longer, Nikola Tesla or Milutin Milankovic?"]

start_time = time.time()
outputs = fast_inference(input_texts)
end_time = time.time()

elapsed_time = (end_time - start_time) * 1000
print("Outputs:", outputs)
print(f"Elapsed time: {elapsed_time:.4f} milliseconds")

c:\Users\shaheryar.ahmed\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shaheryar.ahmed\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Outputs: ['When did Nikola Tesla die. When did Milutin Milankovic die ']
Elapsed time: 78.7923 milliseconds
